# [Module 2.1] 훈련 아테펙트를 Re-Packaging 하기

이 노트북은 아래와 같은 작업을 합니다. 

---



In [ ]:
%store -r artifact_path

---
# 1. 패키징 모델
- 패키징할 폴더 생성
- 훈련된 모델 아티펙트 다운로드 및 해제
- 모델 아티펙트, 추론 코드의 폴더 지정 후에 패키징 폴더에 복사
- 패키징 폴더를 model.tar.gz 압축 후에 S3에 업로딩
- 



In [ ]:
import os

package_dir = 'model_pkg'
os.makedirs(package_dir, exist_ok=True)



추론 코드가 있는 폴더

In [ ]:
code_dir = '../../../src/*'

In [ ]:
%%sh -s {artifact_path} {package_dir} {code_dir}

artifact_path=$1
package_dir=$2
code_dir=$3

cd $package_dir # 폴더 생성
echo $PWD
rm -rf ./*
aws s3 cp $artifact_path . # 아티펙트 다운로드
tar -xzvf model.tar.gz # 압축 해제
rm model.tar.gz # 압축 파일 삭제
mkdir -p code # code 폴더 생성
cp -r $code_dir code/ # src 파일 모두 카피
tar -czvf model.tar.gz * # model.tar.gz 파일 생성

# 2. S3 에 업로드

In [ ]:
import sagemaker
sagemaker_session  = sagemaker.session.Session()
prefix='ncf/inference'
bucket = sagemaker.session.Session().default_bucket()
print("bucket: ", bucket)

model_path = os.path.join(package_dir, 'model.tar.gz')
byom_artifact = sagemaker_session.upload_data(model_path, bucket, prefix)
print("byom_artifact: \n", byom_artifact)

In [ ]:
%store byom_artifact